# Testing the xbar models

First load in the model architecture.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import pickle

class DEHNNLayer(nn.Module):
    def __init__(self, node_in_features, edge_in_features, vn_features, hidden_features):
        super(DEHNNLayer, self).__init__()
        self.node_mlp1 = nn.Sequential(nn.Linear(edge_in_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, edge_in_features))
        
        self.edge_mlp2 = nn.Sequential(nn.Linear(node_in_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, node_in_features))
        
        self.edge_mlp3 = nn.Sequential(nn.Linear(2 * node_in_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, 2 * node_in_features))

        self.node_to_virtual_mlp = nn.Sequential(nn.Linear(node_in_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, vn_features))
        
        self.virtual_to_higher_virtual_mlp = nn.Sequential(nn.Linear(vn_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, vn_features))
        
        self.higher_virtual_to_virtual_mlp = nn.Sequential(nn.Linear(vn_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, vn_features))
        
        self.virtual_to_node_mlp = nn.Sequential(nn.Linear(vn_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, edge_in_features))


    def forward(self, node_features, edge_features, vn_features, super_vn_features, hypergraph):

        # Node Update
        transformed_edge_features = self.node_mlp1(edge_features)
        updated_node_features = torch.matmul(hypergraph.incidence_matrix, transformed_edge_features)

        # Edge Update
        transformed_node_features = self.edge_mlp2(node_features)
        driver_features = torch.matmul(hypergraph.driver_matrix, transformed_node_features)
        sink_features = torch.matmul(hypergraph.sink_matrix, transformed_node_features)
        updated_edge_features = torch.cat([driver_features, sink_features], dim=1)
        updated_edge_features = self.edge_mlp3(updated_edge_features)
        
        # First Level VN Update
        node_to_virtual_features = self.node_to_virtual_mlp(node_features)
        updated_vn_features = torch.matmul(hypergraph.vn_matrix, node_to_virtual_features)
        updated_vn_features += self.higher_virtual_to_virtual_mlp(super_vn_features)

        # Top Level VN Update
        virtual_to_higher_virtual_features = self.virtual_to_higher_virtual_mlp(vn_features)
        updated_super_vn_features = torch.sum(virtual_to_higher_virtual_features, dim=0)

        # VN to node update
        virtual_to_node_features = self.virtual_to_node_mlp(vn_features)
        propagated_features = torch.matmul(hypergraph.vn_matrix.T, virtual_to_node_features)
        updated_node_features += propagated_features

        return updated_node_features, updated_edge_features, updated_vn_features, updated_super_vn_features


class DEHNN(nn.Module):
    def __init__(self, num_layers, node_in_features, edge_in_features, hidden_features=24):
        super(DEHNN, self).__init__()
        self.num_layers = num_layers
        self.layers = nn.ModuleList()
        
        # Create multiple layers for DEHNN
        vn_in_features = node_in_features
        for i in range(num_layers):
            self.layers.append(DEHNNLayer(node_in_features, edge_in_features, vn_in_features, hidden_features))
            node_in_features, edge_in_features = edge_in_features, node_in_features
            edge_in_features *= 2
        
        self.output_layer = nn.Sequential(nn.Linear(edge_in_features, hidden_features),
                                       nn.ReLU(),
                                       nn.Linear(hidden_features, 1))

    def forward(self, node_features, edge_features, vn_features, super_vn_features, hypergraph):
        # Pass through each layer
        for layer in self.layers:
            node_features, edge_features, vn_features, super_vn_features = layer(node_features, edge_features, vn_features, super_vn_features, hypergraph)
        
        # Output prediction for nodes
        output = self.output_layer(edge_features)
        return output[:,0]

class Hypergraph:
    def __init__(self, incidence_matrix, driver_matrix, sink_matrix, vn_matrix):
        self.incidence_matrix = incidence_matrix
        self.driver_matrix = driver_matrix
        self.sink_matrix = sink_matrix
        self.vn_matrix = vn_matrix

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

clean_data_dir = '../../data/chips/clean_data/'

train_idx = [1, 2, 3, 4, 5, 6, 7, 8, 11, 12]
data = []

for i in train_idx:
    connectivity = np.load(clean_data_dir + str(i) + '.connectivity.npz')
    incidence_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([connectivity['row'], connectivity['col']])), torch.ones(connectivity['dirs'].shape), dtype=torch.float).to(device)

    drivers = np.load(clean_data_dir + str(i) + '.drivers.npz')
    driver_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([drivers['col'], drivers['row']])), torch.ones(drivers['data'].shape), dtype=torch.float).to(device)

    sinks = np.load(clean_data_dir + str(i) + '.sinks.npz')
    sink_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([sinks['col'], sinks['row']])), torch.ones(sinks['data'].shape), dtype=torch.float).to(device)

    features = np.load(clean_data_dir + str(i) + '.features.npz')
    node_features = features['node_features']
    edge_features = features['net_features']
    wire = features['hpwl']
    # congestion = np.argmax(congestion, axis=1)

    node_features = torch.tensor(node_features, dtype=torch.float).to(device)
    edge_features = torch.tensor(edge_features, dtype=torch.float).to(device)
    # congestion = torch.tensor(congestion, dtype=torch.float).to(device)
    wire = torch.tensor(wire, dtype=torch.float).to(device)

    num_nodes, num_node_features = node_features.shape
    num_edges, num_edge_features = edge_features.shape

    virtual_nodes = np.load(clean_data_dir + str(i) + '.virtual_nodes.npz')
    vn_rows = virtual_nodes['row']
    vn_cols = virtual_nodes['col']
    vn_matrix = torch.sparse_coo_tensor(torch.tensor(np.array([vn_rows, vn_cols])), torch.ones(len(vn_rows)), dtype=torch.float).to(device)

    num_vn = vn_matrix.shape[0]
    num_vn_features = num_node_features
    vn_features = torch.zeros((num_vn, num_vn_features), dtype=torch.float).to(device)
    super_vn_features = torch.zeros(num_vn_features, dtype=torch.float).to(device)

    hypergraph = Hypergraph(incidence_matrix, driver_matrix, sink_matrix, vn_matrix)

    data.append((node_features, edge_features, vn_features, super_vn_features, hypergraph, wire))


Load in the model weights

In [ ]:
model = torch.load('hpwl_xbar.pt')

Open the chip you want to test by index

In [ ]:
node_features, edge_features, vn_features, super_vn_features, hypergraph, wire = data[0]

Run the chip through the model

In [ ]:
output = model(node_features, edge_features, vn_features, super_vn_features, hypergraph)

Calculate metrics from the outputted data

In [ ]:
prediction = output.detach().cpu().numpy()
actual = wire.detach().cpu().numpy()

mae = np.mean(np.abs(prediction - actual))
print(f'MAE: {mae}')

rmse = np.mean((prediction - actual) ** 2)
print(f'RMSE: {rmse}')